In [1]:
import numpy as np
import networkx as nx

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric_temporal.nn.recurrent import A3TGCN2
from torch_geometric_temporal.signal import temporal_signal_split
import pandas as pd
import json

# GPU support
DEVICE = torch.device('cpu') # cuda
shuffle=True
batch_size = 32

ModuleNotFoundError: No module named 'torch_geometric_temporal'

In [ ]:
lines = pd.read_csv(r'C:\Users\Dell\Documents\GitHub\pinns_opf\data\interim\lines_5.csv')
nodes = pd.read_csv(r'C:\Users\Dell\Documents\GitHub\pinns_opf\data\interim\nodes_5.csv')

In [ ]:
cons_id = 0
cons_id_node = {}

pv_id = 0
pv_id_node = {}

ev_id = 0
ev_id_node = {}

dg_id = 0
dg_id_node = {}

ess_id = 0
ess_id_node = {}
for row in nodes.iterrows():

    for j in range(len(row[1])):

        if row[1][j] == 'cons':
            cons_id += 1
            cons_id_node[cons_id] = row[1][0]
        
        if row[1][j] == 'pv':
            pv_id += 1
            pv_id_node[pv_id] = row[1][0]
        
        if row[1][j] == 'ev':
            ev_id += 1
            ev_id_node[ev_id] = row[1][0]

        if row[1][j] == 'ess':
            ess_id += 1
            ess_id_node[ess_id] = row[1][0]
        
        if row[1][j] == 'dg':
            dg_id += 1
            dg_id_node[dg_id] = row[1][0]


In [ ]:
with open('variable_data.json') as f:
    dataset = json.load(f)

In [ ]:
for key in dataset.keys():

    for node_id in dataset[key].keys():

        if 'V' in key:
            print(node_id)

1
2
3
4
5


In [ ]:
hetero_graph_dataset = dict()
node_id = nodes['Nodes'].values
for node in node_id:
    hetero_graph_dataset[node] = []
    
for key in dataset.keys():

    for node_id in dataset[key].keys():
        
        if 'cons' in key:
            node_id_g = cons_id_node[int(node_id)]
            hetero_graph_dataset[node_id_g].append(dataset[key][str(node_id_g)])
        elif 'ev' in key:
            node_id_g = ev_id_node[int(node_id)]
            hetero_graph_dataset[node_id_g].append(dataset[key][str(node_id_g)])
        elif 'pv' in key:
            node_id_g = pv_id_node[int(node_id)]
            hetero_graph_dataset[node_id_g].append(dataset[key][str(node_id)])
        elif 'ess' in key:
            node_id_g = ess_id_node[int(node_id)]
            hetero_graph_dataset[node_id_g].append(dataset[key][str(node_id)])
        elif 'dg' in key:
            node_id_g = dg_id_node[int(node_id)]
            hetero_graph_dataset[node_id_g].append(dataset[key][str(node_id)])
        elif 'V' in key:
            hetero_graph_dataset[int(node_id)].append(dataset[key][str(node_id)])


In [ ]:
list_ = []
for node,value in hetero_graph_dataset.items():
    hetero_graph_dataset[node] = np.array(value)
    list_.append(np.array(value))
    print(node,hetero_graph_dataset[node].shape)

1 (9, 24)
2 (9, 24)
3 (9, 24)
4 (9, 24)
5 (9, 24)


In [ ]:
# Stack the arrays from the dictionary into the result array
# Create an empty numpy array of size 24x5x9
result_array = np.empty((24, 5, 9))

for i, key in enumerate(hetero_graph_dataset.keys()):
    result_array[:, i, :] = hetero_graph_dataset[key].T

In [ ]:
# Reshape the array to (120, 9) to apply normalization to each feature
data_reshaped = np.reshape(result_array, (result_array.shape[0]*result_array.shape[1], result_array.shape[2]))

# Calculate the mean and standard deviation along the 0th axis (rows)
mean = np.mean(data_reshaped, axis=0)
std = np.std(data_reshaped, axis=0)

# Normalize the data using mean and standard deviation
normalized_data = (data_reshaped - mean) / std

# Reshape the normalized data back to the original shape
normalized_data = np.reshape(normalized_data, (result_array.shape[0], result_array.shape[1], result_array.shape[2]))
normalized_data[:,:,-1] = result_array[:,:,-1]

In [ ]:
# import numpy as np

# # Assuming your array is called 'data'
# # Shape: (24, 5, 9)

# # Reshape the array to (120, 9) to apply normalization to each feature
# data_reshaped = np.reshape(result_array, (result_array.shape[0]*result_array.shape[1], result_array.shape[2]))

# # Calculate the minimum and maximum values along the 0th axis (rows)
# min_vals = np.min(data_reshaped, axis=0)
# max_vals = np.max(data_reshaped, axis=0)

# # Normalize the data using min-max normalization
# normalized_data = (data_reshaped - min_vals) / (max_vals - min_vals)

# # Reshape the normalized data back to the original shape
# normalized_data = np.reshape(normalized_data, (result_array.shape[0], result_array.shape[1], result_array.shape[2]))

In [ ]:
edge_index = np.array([lines['From'].values,lines['To'].values])


In [ ]:
edge_weigth = np.ones(edge_index.shape[1])

In [ ]:
features = normalized_data[:-1,:,:]
targets = normalized_data[1:,:,:]

In [ ]:
features_seq = []
targets_seq = []

for i in range(1000*len(features[:,1,1])):
    features_seq.append(features[i%23,:,:])
    targets_seq.append(targets[i%23,:,:])

In [ ]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

dataset = StaticGraphTemporalSignal(
    edge_index=edge_index, edge_weight=edge_weigth,
    features=features_seq, targets=targets_seq
)

In [ ]:
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [ ]:
dataset.__dict__['edge_weight']

array([1., 1., 1., 1., 1., 1., 1.])

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN,GCLSTM

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 9)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [ ]:
from torch_geometric_temporal.nn.recurrent import GConvLSTM

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GConvLSTM(node_features, 32, 1, normalization='rw')
        self.linear = torch.nn.Linear(32, 9)

    def forward(self, x, edge_index, edge_weight, h, c):
        h_0, c_0 = self.recurrent(x, edge_index, edge_weight, h, c)
        h = F.relu(h_0)
        h = self.linear(h)
        return h, h_0, c_0
        
model = RecurrentGCN(node_features=9)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
from tqdm import tqdm

model = RecurrentGCN(node_features=9)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
model.train()

for epoch in tqdm(range(200)):
    cost = 0
    h, c = None, None
    for time, snapshot in enumerate(train_dataset):
        y_hat, h, c = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, h, c)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)     
    cost = cost / (time+1)
    print(cost)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

  0%|          | 0/200 [00:00<?, ?it/s]

tensor(1.0314, grad_fn=<DivBackward0>)


  0%|          | 1/200 [05:41<18:52:02, 341.32s/it]

tensor(0.9613, grad_fn=<DivBackward0>)


  1%|          | 2/200 [10:08<16:22:18, 297.67s/it]

...

  3%|▎         | 6/200 [26:24<13:27:22, 249.70s/it]

tensor(0.6987, grad_fn=<DivBackward0>)


  4%|▎         | 7/200 [30:28<13:17:17, 247.86s/it]

tensor(0.6437, grad_fn=<DivBackward0>)


  4%|▍         | 8/200 [34:28<13:04:29, 245.15s/it]